<a href="https://colab.research.google.com/github/othmbela/gotham-network-packet-labeller/blob/main/notebooks/GothamDataset2025%20--%20Data%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gotham Dataset 2025

## Loading the datasets

### Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np

### Import data from Google drive

The dataset can be imported using google drive. Import drive and use mount keyword to make drive as active directory. Variable basedir stores the location of folder where dataset is stored in the drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change this line your folder where the data is found
basedir = '/content/drive/MyDrive/GothamDataset2025/'

In [ ]:
df = pd.read_csv(os.path.join(basedir,'iotsim-air-quality-1.csv'))

## Dataset Description and Exploration

The dataset is explored more here and some information about the dataset is described. Some functions for instance .info(), .isna() etc are used. This section will include how we can manipulate the dataset to how to access specific values/attributes from the dataset.

Have an initial inspection of the data

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df.describe(include=[int, float])

In [ ]:
df.describe(include=[object]).transpose()

In [ ]:
df.label.value_counts()

### Dealing with features with quasi null std deviation

Standard deviation denoted by sigma (σ) is the average of the squared root differences from the mean.

In [ ]:
df_std = df.std(numeric_only=True)
df_std

In [ ]:
# Find Features that meet the threshold
constant_features = [column for column, std in df_std.items() if std < 0.01]

# Drop the constant features
df.drop(labels=constant_features, axis=1, inplace=True)

## Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import glob
import gc

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    FunctionTransformer,
    StandardScaler,
    OneHotEncoder,
    LabelEncoder,
)

In [ ]:
def extract_protocols_and_ports(df):
    """
    Extract protocol and port information from packet data.

    Parameters:
    df (pandas.DataFrame): Input dataset.

    Returns:
    pandas.DataFrame: Dataset with extracted protocol and port information.
    """
    src_ports, dst_ports, protocols = [], [], []
    for _, pkt in df.iterrows():
        if ":tcp" in pkt["frame.protocols"]:
            protocol = "TCP"
            src_port = int(pkt["tcp.srcport"])
            dst_port = int(pkt["tcp.dstport"])
        elif ":udp" in pkt["frame.protocols"]:
            protocol = "UDP"
            src_port = int(pkt["udp.srcport"])
            dst_port = int(pkt["udp.dstport"])
        elif ":icmp" in pkt["frame.protocols"]:
            protocol = "ICMP"
            src_port = np.nan
            dst_port = np.nan

        protocols.append(protocol)
        src_ports.append(src_port)
        dst_ports.append(dst_port)

    df["ip.protocol"] = protocols
    df["src.port"] = src_ports
    df["dst.port"] = dst_ports

    df.drop(
        columns=[
            "ip.proto",
            "tcp.srcport",
            "tcp.dstport",
            "udp.srcport",
            "udp.dstport",
        ],
        axis=1,
        inplace=True,
    )

    return df

def convert_time(df):
    """
    Convert frame time to Unix timestamp.

    Parameters:
    df (pandas.DataFrame): Input dataset.

    Returns:
    pandas.DataFrame: Dataset with timestamp column.
    """
    df["frame.time"] = df["frame.time"].str.replace("  ", " ")
    df["frame.time"] = df["frame.time"].str.replace(" BST", "")
    df["frame.time"] = pd.to_datetime(
        df["frame.time"], format="%b %d, %Y %H:%M:%S.%f000"
    )
    df["timestamp"] = df["frame.time"].values.astype(np.int64) // 10**9

    return df

def convert_ports(df, port_hierarchy_map_iot):
    """
    Convert ports to categorical values using a predefined port hierarchy.

    Parameters:
    df (pandas.DataFrame): Input dataset.

    Returns:
    pandas.DataFrame: Dataset with converted ports.
    """
    def port_to_categories(port, port_hierarchy_map_iot):
      """Convert port number to category according to port_map."""
      for p_range, p_name in port_hierarchy_map_iot:
          if port in p_range:
              return p_name

      return ""

    df["src.port"] = df["src.port"].apply(lambda port: port_to_categories(port, port_hierarchy_map_iot))
    df["dst.port"] = df["dst.port"].apply(lambda port: port_to_categories(port, port_hierarchy_map_iot))

    return df

def convert_checksums(df):
    """
    Convert checksum fields to integers, replacing missing values with a default value.

    Parameters:
    df (pandas.DataFrame): Input dataset.

    Returns:
    pandas.DataFrame: Dataset with converted checksum fields.
    """
    df["ip.checksum"] = df["ip.checksum"].apply(lambda x: int(str(x), 16) if pd.notna(x) else 0)
    df["tcp.checksum"] = df["tcp.checksum"].apply(lambda x: int(str(x), 16) if pd.notna(x) else 0)
    df["tcp.options"] = df["tcp.options"].apply(lambda x: int(str(x), 16) if pd.notna(x) else 0).astype(float)

    return df

def fill_missing_values(df):
    """
    Fill missing values with a default value (-1).

    Parameters:
    df (pandas.DataFrame): Input dataset.

    Returns:
    pandas.DataFrame: Dataset with missing values filled.
    """
    num_cols = df.select_dtypes(include=["number"]).columns
    df[num_cols] = df[num_cols].fillna(-1)

    cat_cols = df.select_dtypes(exclude=["number"]).columns
    df[cat_cols] = df[cat_cols].fillna(-1)

    return df

def group_labels(df):
    """
    Group the attack labels into broader categories.
    """
    attack_group = {
        "Normal": "Normal",
        "TCP Scan": "Network Scanning",
        "UDP Scan": "Network Scanning",
        "Telnet Brute Force": "Brute Force",
        "Reporting": "Infection",
        "Ingress Tool Transfer": "Infection",
        "File Download": "Infection",
        "CoAP Amplification": "DoS",
        "Merlin TCP Flooding": "DoS",
        "Merlin UDP Flooding": "DoS",
        "Merlin ICMP Flooding": "DoS",
        "Merlin C&C Communication": "C&C Communication",
        "Mirai TCP Flooding": "DoS",
        "Mirai UDP Flooding": "DoS",
        "Mirai GRE Flooding": "DoS",
        "Mirai C&C Communication": "C&C Communication",
    }

    # Create grouped label column
    df["label_category"] = df["label"].map(lambda x: attack_group.get(x, "Other"))
    return df

def select_columns(df):
    """
    Select relevant columns from the dataset.

    Parameters:
    df (pandas.DataFrame): Input dataset.

    Returns:
    pandas.DataFrame: Dataset with selected columns.
    """
    selected_columns = [
        "timestamp", "frame.len", "ip.protocol", "src.port",
        "dst.port", "ip.flags", "ip.ttl", "ip.checksum",
        "tcp.flags", "tcp.window_size_value", "tcp.window_size_scalefactor", "tcp.checksum",
        "tcp.options", "tcp.pdu.size", "label", "label_category",
    ]

    df = df[selected_columns]
    df = df[df["label"] != "Unknown"]
    return df

def unpack_flags(X):
    X = X.copy()

    # Unpack IP flags
    ip_flags = X["ip.flags"].apply(lambda x: int(x, 16)).values.astype(np.uint8)
    ip_flags = np.unpackbits(ip_flags.reshape((-1, 1)), axis=1, bitorder="little")[:, :3]

    # Unpack TCP flags (handle -1 for missing values)
    tcp_flags = X["tcp.flags"]\
        .apply(lambda x: int(x, 16) if x != -1 else 0)\
        .values.astype(np.uint8)

    tcp_flags = np.unpackbits(tcp_flags.reshape((-1, 1)), axis=1, bitorder="little")[:, :9]

    # Combine flags into a single output array
    return np.hstack([ip_flags, tcp_flags])

In [ ]:
VALIDATION_SIZE, TESTING_SIZE = 0.2, 0.2

OUTPUT_DIR = ""

GLOBAL_CATEGORICAL_VALUES = pd.DataFrame({
    "ip.protocol": ['TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP', 'UDP', 'ICMP', 'TCP'],
    "src.port": ['mqttPorts', 'coapPorts', 'rtspPorts', 'httpPorts', 'mailPorts', 'dnsPorts', 'ftpPorts', 'shellPorts', 'remoteExecPorts', 'authPorts', 'passwordPorts', 'newsPorts', 'chatPorts', 'printPorts', 'timePorts', 'dbmsPorts', 'dhcpPorts', 'whoisPorts', 'netbiosPorts', 'kerberosPorts', 'RPCPorts', 'snmpPorts', 'PRIVILEGED_PORTS', 'NONPRIVILEGED_PORTS', ''],
    "dst.port": ['mqttPorts', 'coapPorts', 'rtspPorts', 'httpPorts', 'mailPorts', 'dnsPorts', 'ftpPorts', 'shellPorts', 'remoteExecPorts', 'authPorts', 'passwordPorts', 'newsPorts', 'chatPorts', 'printPorts', 'timePorts', 'dbmsPorts', 'dhcpPorts', 'whoisPorts', 'netbiosPorts', 'kerberosPorts', 'RPCPorts', 'snmpPorts', 'PRIVILEGED_PORTS', 'NONPRIVILEGED_PORTS', ''],
})

GLOBAL_LABEL_VALUES = pd.DataFrame({
    "label": ["Normal", "Network Scanning", "Brute Force", "Infection", "C&C Communication", "DoS"]
})

PORT_HIERARCHY_MAP_IOT = [
    ([1883, 8883], "mqttPorts"),
    ([5683, 5684], "coapPorts"),
    ([8554, 8322, 8000, 8001, 8002, 8003, 1935, 8888], "rtspPorts"),
    ([80, 280, 443, 591, 593, 777, 488, 1183, 1184, 2069, 2301, 2381, 8008, 8080], "httpPorts"),
    ([24, 25, 50, 58, 61, 109, 110, 143, 158, 174, 209, 220, 406, 512, 585, 993, 995], "mailPorts"),
    ([42, 53, 81, 101, 105, 261], "dnsPorts"),
    ([20, 21, 47, 69, 115, 152, 189, 349, 574, 662, 989, 990], "ftpPorts"),
    ([22, 23, 59, 87, 89, 107, 211, 221, 222, 513, 614, 759, 992], "shellPorts"),
    ([512, 514], "remoteExecPorts"),
    ([13, 56, 113, 316, 353, 370, 749, 750], "authPorts"),
    ([229, 464, 586, 774], "passwordPorts"),
    ([114, 119, 532, 563], "newsPorts"),
    ([194, 258, 531, 994], "chatPorts"),
    ([35, 92, 170, 515, 631], "printPorts"),
    ([13, 37, 52, 123, 519, 525], "timePorts"),
    ([65, 66, 118, 150, 156, 217], "dbmsPorts"),
    ([546, 547, 647, 847], "dhcpPorts"),
    ([43, 63], "whoisPorts"),
    (range(137, 139 + 1), "netbiosPorts"),
    ([88, 748, 750], "kerberosPorts"),
    ([111, 121, 369, 530, 567, 593, 602], "RPCPorts"),
    ([161, 162, 391], "snmpPorts"),
    (range(0, 1024), "PRIVILEGED_PORTS"),
    (range(1024, 65536), "NONPRIVILEGED_PORTS")
]

In [ ]:
filenames = glob.glob(os.path.join(basedir, "*.csv"))

for filename in filenames:

    print(f"Processing {filename}")

    processed_chunks = []  # List to hold processed data chunks

    # Process CSV file in chunks for memory efficiency
    for chunk in pd.read_csv(filename, sep=",", low_memory=False, chunksize=10000):
        chunk = extract_protocols_and_ports(chunk)
        chunk = convert_time(chunk)
        chunk = convert_ports(chunk, PORT_HIERARCHY_MAP_IOT)
        chunk = convert_checksums(chunk)
        chunk = fill_missing_values(chunk)
        chunk = group_labels(chunk)
        chunk = select_columns(chunk)

        processed_chunks.append(chunk)

    # Concatenate all processed chunks into a single DataFrame
    df = pd.concat(processed_chunks, ignore_index=True)

    # Split data into training, validation, and testing sets

    labels = df["label_category"]
    features = df.drop(labels=["label", "label_category"], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
        features,
        labels,
        test_size=(VALIDATION_SIZE + TESTING_SIZE),
        random_state=42,
        stratify=labels,
    )
    X_test, X_val, y_test, y_val = train_test_split(
        X_test,
        y_test,
        test_size=TESTING_SIZE / (VALIDATION_SIZE + TESTING_SIZE),
        random_state=42,
    )

    # Scale the data and split it into features and labels
    flags_features = ["ip.flags", "tcp.flags"]
    categorical_features = ["ip.protocol", "src.port", "dst.port"]
    numeric_features = ['timestamp', 'frame.len', 'ip.ttl', 'ip.checksum',
                        'tcp.window_size_value', 'tcp.window_size_scalefactor', 'tcp.checksum',
                        'tcp.options', 'tcp.pdu.size']
    preprocessor = ColumnTransformer(
        transformers=[
            ("flags", FunctionTransformer(unpack_flags), flags_features),
            ("categoricals", OneHotEncoder(drop="first", sparse_output=True, handle_unknown="error"), categorical_features),
            ("numericals", StandardScaler(), numeric_features),
        ]
    )

    preprocessor.fit(X_train)
    preprocessor["categoricals"].fit(GLOBAL_CATEGORICAL_VALUES)

    # Preprocess the features
    X_train = pd.DataFrame(preprocessor.transform(X_train))
    X_val = pd.DataFrame(preprocessor.transform(X_val))
    X_test = pd.DataFrame(preprocessor.transform(X_test))
    print(X_train.shape)

    # Preprocess the labels
    le = LabelEncoder()
    le.fit(GLOBAL_LABEL_VALUES)

    y_train = pd.DataFrame(le.transform(y_train), columns=["label"])
    y_val = pd.DataFrame(le.transform(y_val), columns=["label"])
    y_test = pd.DataFrame(le.transform(y_test), columns=["label"])

    # Extract IoT device name from filename for saving
    iot_device = os.path.basename(filename).rstrip(".csv")

    # Save the processed data to pickle files for each IoT device
    X_train.to_pickle(os.path.join(basedir, "processed", f"{iot_device}_train_features.pkl"), compression="gzip")
    y_train.to_pickle(os.path.join(basedir, "processed", f"{iot_device}_train_labels.pkl"), compression="gzip",)

    X_val.to_pickle(os.path.join(basedir, "processed", f"{iot_device}_val_features.pkl"), compression="gzip")
    y_val.to_pickle(os.path.join(basedir, "processed", f"{iot_device}_val_labels.pkl"), compression="gzip")

    X_test.to_pickle(os.path.join(basedir, "processed", f"{iot_device}_test_features.pkl"), compression="gzip")
    y_test.to_pickle(os.path.join(basedir, "processed", f"{iot_device}_test_labels.pkl"), compression="gzip")

    # Clean up memory by deleting intermediate variables and performing garbage collection
    del (
        chunk,
        # df,
        X_train,
        y_train,
        X_val,
        y_val,
        X_test,
        y_test,
    )
    gc.collect()